In [6]:
# Set up data collection foundation

CREATURE_NAME=[]
CREATURE_CAT=[]
SEASON_EP=[]
SHOW_NAME=[]

TABLE_COLUMNS = ['creature_name', 'creature_cat', 'season_ep', 'show_name']

DATABASE = {'creature_name':CREATURE_NAME, 'creature_cat':CREATURE_CAT, 'season_ep':SEASON_EP, 'show_name':SHOW_NAME}

SUPERNATURAL_URL = 'http://supernatural.wikia.com/wiki/Category:Creatures?page='

categoryLinks = []
creatureLinks = []

In [ ]:
# Libraries

from bs4 import BeautifulSoup as soup
import pandas as pd
import re
from urllib import request

In [ ]:
# Convert html to a more user friendly format

def soupTheLink(url):
    html = request.urlopen(url).read().decode('utf8')
    return soup(html, 'lxml')

In [ ]:
# Collect a list of links to the categories of creatures

def collectCatLinks(url):
    souped = soupTheLink(url)

    for subcategories in souped.find_all(id='mw-subcategories'):
        for gallery in subcategories.find_all(class_='category-gallery-room1'):
            for listItem in gallery.find_all(class_='category-gallery-item'):
                link = listItem.find('a', href = True)
                try:
                    fullLink = link['href']
                    categoryLinks.append(fullLink)
                except TypeError:
                    pass

In [2]:
# Find the number of pages of each category

def findNumCatPages(url):
    souped = soupTheLink(url)
    pages = 0
    for subcategories in souped.find_all(id='mw-subcategories'):
        for pagination in subcategories.find_all(class_='wikia-paginator'):
            for listItem in pagination.find_all('li'):
                pages += 1
    return pages

In [3]:
numPages = findNumCreaturePages(SUPERNATURAL_URL) - 1
for i in range(1,numPages):
    url = SUPERNATURAL_URL + str(i)
    print(url)
    
    collectCatLinks(url)
    print(categoryLinks)

http://supernatural.wikia.com/wiki/Category:Creatures?page=1
['http://supernatural.wikia.com/wiki/Category:Angels', 'http://supernatural.wikia.com/wiki/Category:Monster', 'http://supernatural.wikia.com/wiki/Category:Strongest_of_Species', 'http://supernatural.wikia.com/wiki/Category:Deities', 'http://supernatural.wikia.com/wiki/Category:Demons', 'http://supernatural.wikia.com/wiki/Category:One-Episode_Species', 'http://supernatural.wikia.com/wiki/Category:Higher_Beings', 'http://supernatural.wikia.com/wiki/Category:Hybrid', 'http://supernatural.wikia.com/wiki/Category:Reaper', 'http://supernatural.wikia.com/wiki/Category:Constructs', 'http://supernatural.wikia.com/wiki/Category:Ghosts', 'http://supernatural.wikia.com/wiki/Category:God%27s_creations', 'http://supernatural.wikia.com/wiki/Category:Witches', 'http://supernatural.wikia.com/wiki/Category:Shapeshifter', 'http://supernatural.wikia.com/wiki/Category:Amazons', 'http://supernatural.wikia.com/wiki/Category:Fairies']
http://superna

In [ ]:
# Import creature category to database

def findCategory(url):
    category = str(url[44:])
    DATABASE['creature_cat'].append(category)

In [ ]:
# Find the number of pages in each category

def findNumCreaturePages(url):
    souped = soupTheLink(url)
    pages = 0
    for subcategories in souped.find_all(id='mw-pages'):
        for pagination in subcategories.find_all(class_='wikia-paginator'):
            for listItem in pagination.find_all('li'):
                pages += 1
    return pages

In [9]:
# Compile a list of links to each individual creature

def collectCreatureLinks():
    for mainLink in categoryLinks:
        
        def collectLinks(url):
            souped = soupTheLink(url)

            for subcategories in souped.find_all(id='mw-pages'):
                for listItem in subcategories.find_all(class_='category-gallery-item'):
                    link = listItem.find('a', href = True)
                    try:
                        fullLink = link['href']
                        creatureLinks.append(fullLink)
                    except TypeError:
                        pass
                    findCategory(mainLink)
                return creatureLinks

        numPages = findNumCreaturePages(mainLink) - 1
        for i in range(1,numPages):
            url = mainLink  + '?page=' + str(i)
            print(url)

            collectLinks(url)
            print(creatureLinks)

collectCreatureLinks()

http://supernatural.wikia.com/wiki/Category:Angels?page=1
['http://supernatural.wikia.com/wiki/Castiel', 'http://supernatural.wikia.com/wiki/Lucifer', 'http://supernatural.wikia.com/wiki/Gabriel', 'http://supernatural.wikia.com/wiki/Michael', 'http://supernatural.wikia.com/wiki/Angels', 'http://supernatural.wikia.com/wiki/Metatron', 'http://supernatural.wikia.com/wiki/Archangels', 'http://supernatural.wikia.com/wiki/Gadreel', 'http://supernatural.wikia.com/wiki/Seraphim', 'http://supernatural.wikia.com/wiki/Anna_Milton', 'http://supernatural.wikia.com/wiki/Raphael', 'http://supernatural.wikia.com/wiki/Balthazar', 'http://supernatural.wikia.com/wiki/Reapers', 'http://supernatural.wikia.com/wiki/Hannah', 'http://supernatural.wikia.com/wiki/Nephilim', 'http://supernatural.wikia.com/wiki/Tessa', 'http://supernatural.wikia.com/wiki/Ezekiel', 'http://supernatural.wikia.com/wiki/Naomi', 'http://supernatural.wikia.com/wiki/Uriel', 'http://supernatural.wikia.com/wiki/Zachariah', 'http://superna

In [8]:
# Import the rest of the data to the database

def intoCreatureLinks(listOfCreatureLinks):
    print("number of creatures = ", len(listOfCreatureLinks))
    count = 0
    for url in listOfCreatureLinks:
        count += 1
        print('creature number ', count)
        souped = soupTheLink(url)
        
        #collect creature name
        name = url[35:]
        DATABASE['creature_name'].append(name)
        
        #collect show name
        DATABASE['show_name'].append('Supernatural')

        #collect season appearances
        currentSeason = 0
        seasonLink = ''
        for body in souped.find_all(id='mw-content-text'):
            SEASON_EP=[]
            for lists in body.find_all('ul'):
                for bullet in lists.find_all('li'):
                    for seasons in bullet.find_all('a', href = True):
                        seasonDegree=[]
                        try:
                            season = seasons['href']
                            seasonLink = str(season)
                            if seasonLink[:13] == '/wiki/Season_':
                                
                                
                                try:
                                    if int(seasonLink[13:]) > currentSeason:
                                        currentSeason = int(seasonLink[13:])
                                        seasonDegree.append(currentSeason)

                                        for epList in bullet.find_all('ul'):
                                            epCount = 0
                                            for episodes in epList.find_all('li'):
                                                epCount += 1

                                            seasonDegree.append(epCount)
                                        SEASON_EP.append(seasonDegree)
                                    else:
                                        break
                                except ValueError:
                                    SEASON_EP.append(['None'])                                
                                
                        except TypeError:
                            pass

            if SEASON_EP == []:
                DATABASE['season_ep'].append('None')   
            else:
                DATABASE['season_ep'].append(SEASON_EP)   




compileData(creatureLinks)
print(DATABASE)

number of creatures =  456
creature number  1
creature number  2
creature number  3
creature number  4
creature number  5
creature number  6
creature number  7
creature number  8
creature number  9
creature number  10
creature number  11
creature number  12
creature number  13
creature number  14
creature number  15
creature number  16
creature number  17
creature number  18
creature number  19
creature number  20
creature number  21
creature number  22
creature number  23
creature number  24
creature number  25
creature number  26
creature number  27
creature number  28
creature number  29
creature number  30
creature number  31
creature number  32
creature number  33
creature number  34
creature number  35
creature number  36
creature number  37
creature number  38
creature number  39
creature number  40
creature number  41
creature number  42
creature number  43
creature number  44
creature number  45
creature number  46
creature number  47
creature number  48
creature number  49
cr

In [11]:
# Ensure that database lists are of the same length

print('1: ', len(CREATURE_NAME), '2: ', len(CREATURE_CAT), '3: ', len(SEASON_EP), '4: ', len(SHOW_NAME), )

1:  456 2:  456 3:  456 4:  456


In [12]:
# Save dataframe to file

pd.DataFrame(DATABASE).T.reset_index().to_csv('supernaturalCreatures.csv', header=True)